In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
from datetime import datetime, timedelta, timezone
import os
import pandas_ta as ta
from tqdm import tqdm
import time 
import pytz
import os
# os.system('cls' if os.name == 'nt' else 'clear')
from IPython.display import clear_output
# clear_output(wait=True)
import importlib
import BaseFunctions
importlib.reload(BaseFunctions)
from BaseFunctions import *

import StrategyList
importlib.reload(StrategyList)
from StrategyList import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load API credentials from environment variables
API_KEY = os.getenv("BinanceAPI_250502")
API_SECRET = os.getenv("BinanceSecret_250502")

import warnings
warnings.filterwarnings("ignore")

client = Client(API_KEY, API_SECRET) if API_KEY and API_SECRET else Client()

BackTime = "2025-05-01 00:00:00"
# Convert string to naive datetime object
BackTime = datetime.strptime(BackTime, '%Y-%m-%d %H:%M:%S')
# Make it timezone-aware (UTC)
BackTime = BackTime.replace(tzinfo=timezone.utc)
# print(BackTime)
BackTime = 1746478489000

# BackTime = int(BackTime.timestamp() * 1000)  # Convert to milliseconds
Interval = '5m'
current_time = time.time()
pair = 'PARTIUSDT'

BuySellFlag = 'Hold'
BuyCounter = 0
SellCounter = 0

In [ ]:
BackCandles = get_candles_data(pair, Interval, 1000, BackTime, client)
# Metric = calculate_metrics(BackCandles, pair)
SCORE = allstrategiesv2(BackCandles, pair, client, BackTime)

# print(SCORE)

In [ ]:
SCORE['ScoreAll'].value_counts().sort_index()

In [165]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Load your data
df = SCORE.copy()

# Create candlestick trace
candlestick = go.Candlestick(
    # x=df['OpenTime'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Candlestick'
)

# Secondary line for score_all
score_trace = go.Scatter(
    # x=df['date'],
    y=df['ScoreAll'],
    mode='lines+markers',
    name='Score All',
    yaxis='y2'
)

# Layout with dual y-axes
layout = go.Layout(
    title='Candles Chart with Score All',
    yaxis=dict(
        domain=[0, 0.7],
        autorange=True  # Ensure auto range
    ),
    yaxis2=dict(
        domain=[0.75, 1],
        anchor='x',
        title='Score All',
        autorange=True  # Ensure auto range
    ),
    hovermode='x',
)

fig = go.Figure(data=[candlestick, score_trace], layout=layout)

# Additional: 
# To enforce auto adjustment during zoom/pan, Plotly's default behavior should handle this.
# If not, you can add a callback to handle relayout events in a Dash app environment.

fig.show()